# Import Libraries

In [ ]:
import os
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

In [ ]:
os.chdir('/Users/sabbirdelowar/Library/CloudStorage/GoogleDrive-sabbir.d.arnob@gmail.com/My Drive/My codes/bd-climate-risk/')

# Define Paths

In [ ]:
rainfall_tif = "data/interim/mean_annual_rainfall.tif"
districts_shp = "data/aoi/aoi-multipoly.shp"
output_dir = "results/individual-hazards"
os.makedirs(output_dir, exist_ok=True)
output_csv = os.path.join(output_dir, "rainfall_hazard.csv")

# Perform Zonal Statistics

In [ ]:
districts = gpd.read_file(districts_shp)
districts = districts.to_crs("EPSG:4326")
pstats = zonal_stats(
    vectors=districts,
    raster=rainfall_tif,
    stats=['mean'],
    geojson_out=True
)

# Build DataFrame

In [ ]:
district_names = [feature['properties']['name'] for feature in pstats]
mean_rainfall = [feature['properties']['mean'] for feature in pstats]

rainfall_df = pd.DataFrame({
    "district": district_names,
    "mean_annual_rainfall_mm": mean_rainfall
})

# Scale to 0–1

In [ ]:
min_val = rainfall_df['mean_annual_rainfall_mm'].min()
max_val = rainfall_df['mean_annual_rainfall_mm'].max()

rainfall_df['rainfall_hazard_score'] = (
    (rainfall_df['mean_annual_rainfall_mm'] - min_val) / (max_val - min_val)
)

rainfall_df.to_csv(output_csv, index=False)